# ZASHAM AUDIO PROCESSING

In [ ]:
pip install librosa

In [ ]:
pip install yt-dlp

## Download & Process Full Songs

In [3]:
import yt_dlp
import os

download_path = "songs"
os.makedirs(download_path, exist_ok=True)

def download_song(song_name):
    """Download a song from YouTube and force MP3 conversion"""
    ydl_opts = {
        'format': 'bestaudio/best',
        'default_search': 'ytsearch1',
        'noplaylist': True,
        'outtmpl': f'{download_path}/%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'webm',
            'preferredquality': '192',
        }],
        'quiet': False  # You can set this to True if you want cleaner output
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([song_name])
    
    print(f"[INFO] Downloaded and converted: {song_name}")

try:
    with open("song_list.txt", "r") as f:
        songs = [line.strip() for line in f if line.strip()]
except FileNotFoundError:
    print("[ERROR] songs_list.txt not found in the current directory.")
    songs = []

for song in songs:
    try:
        download_song(song)
    except Exception as e:
        print(f"[WARNING] Skipped due to error: {song}")
        print(e)

[generic] Extracting URL: Luther Kendrick Lamar & SZA
[youtube:search] Extracting URL: ytsearch1:Luther Kendrick Lamar & SZA
[download] Downloading playlist: Luther Kendrick Lamar & SZA
[youtube:search] query "Luther Kendrick Lamar & SZA": Downloading web client config
[youtube:search] query "Luther Kendrick Lamar & SZA" page 1: Downloading API JSON
[youtube:search] Playlist Luther Kendrick Lamar & SZA: Downloading 1 items of 1
[download] Downloading item 1 of 1
[youtube] Extracting URL: https://www.youtube.com/watch?v=HfWLgELllZs
[youtube] HfWLgELllZs: Downloading webpage
[youtube] HfWLgELllZs: Downloading tv client config
[youtube] HfWLgELllZs: Downloading player 69f581a5
[youtube] HfWLgELllZs: Downloading tv player API JSON
[youtube] HfWLgELllZs: Downloading ios player API JSON
[youtube] HfWLgELllZs: Downloading m3u8 information
[info] HfWLgELllZs: Downloading 1 format(s): 251
[download] Destination: songs\Kendrick Lamar - luther (Official Audio).webm
[download] 100% of    2.91MiB i

Convert to MP3

In [4]:
import os
import subprocess

input_folder = "songs"  
output_folder = "songs_mp3"
os.makedirs(output_folder, exist_ok=True)  

def convert_webm_to_mp3():
    """Convert all .webm files in a folder to .mp3"""
    for file in os.listdir(input_folder):
        if file.endswith(".webm"):
            input_path = os.path.join(input_folder, file)
            output_path = os.path.join(output_folder, file.replace(".webm", ".mp3"))
            
            
            command = ["ffmpeg", "-i", input_path, "-vn", "-ab", "192k", output_path]
            
            
            try:
                subprocess.run(command, check=True)
                print(f"Converted: {file} -> {output_path}")
            except FileNotFoundError:
                print(f"Error: FFmpeg not found")
                break
            except subprocess.CalledProcessError as e:
                print(f"FFmpeg Error: {e}")
                break

convert_webm_to_mp3()

Converted: Billie Eilish - BIRDS OF A FEATHER (Official Music Video).webm -> songs_mp3\Billie Eilish - BIRDS OF A FEATHER (Official Music Video).mp3
Converted: Chappell Roan - Pink Pony Club (Official Music Video).webm -> songs_mp3\Chappell Roan - Pink Pony Club (Official Music Video).mp3
Converted: Kendrick Lamar - luther (Official Audio).webm -> songs_mp3\Kendrick Lamar - luther (Official Audio).mp3
Converted: Kendrick Lamar - Not Like Us.webm -> songs_mp3\Kendrick Lamar - Not Like Us.mp3
Converted: Kendrick Lamar - tv off (Official Audio).webm -> songs_mp3\Kendrick Lamar - tv off (Official Audio).mp3
Converted: Lady Gaga, Bruno Mars - Die With A Smile (Official Music Video).webm -> songs_mp3\Lady Gaga, Bruno Mars - Die With A Smile (Official Music Video).mp3
Converted: PARTYNEXTDOOR & DRAKE - NOKIA (Official Lyric Video).webm -> songs_mp3\PARTYNEXTDOOR & DRAKE - NOKIA (Official Lyric Video).mp3
Converted: ROSÉ & Bruno Mars - APT. (Official Music Video).webm -> songs_mp3\ROSÉ & Bru

In [5]:
import os
import librosa
import numpy as np
from scipy.ndimage import maximum_filter

def get_audio_files_from_dir(directory, extensions={".wav", ".mp3"}):
    return [
        os.path.join(directory, f)
        for f in os.listdir(directory)
        if os.path.splitext(f)[1].lower() in extensions
    ]

def get_spectrogram_peaks(y, sr, n_fft=4096, hop_length=512, threshold_db=-30):
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    S_db = librosa.amplitude_to_db(S, ref=np.max)
    
    # Local maxima detection
    neighborhood_size = (20, 20)
    local_max = (S_db == maximum_filter(S_db, footprint=np.ones(neighborhood_size)))
    peaks = np.argwhere(local_max & (S_db > threshold_db)) 

    return peaks

def generate_peak_pairs(peaks, fanout=5, time_window=50):
    pairs = []
    for i in range(len(peaks)):
        freq1, t1 = peaks[i]
        for j in range(1, fanout):
            if i + j < len(peaks):
                freq2, t2 = peaks[i + j]
                delta_t = t2 - t1
                if 0 < delta_t <= time_window:
                    pairs.append((freq1, freq2, delta_t, t1))
    return pairs

def process_audio_directory(directory_path):
    all_fingerprints = []
    audio_files = get_audio_files_from_dir(directory_path)

    for audio_path in audio_files:
        try:
            filename = os.path.basename(audio_path)
            y, sr = librosa.load(audio_path, sr=None, mono=True)
            peaks = get_spectrogram_peaks(y, sr)
            peak_pairs = generate_peak_pairs(peaks)

            for (freq1, freq2, delta_t, anchor_time) in peak_pairs:
                all_fingerprints.append((filename, int(freq1), int(freq2), int(delta_t), int(anchor_time)))
        except Exception as e:
            print(f"[ERROR] Skipping {audio_path}: {e}")

    return all_fingerprints

In [6]:
directory_path = "songs_mp3" 

print(f"Processing songs in: {directory_path}")
all_fingerprints = process_audio_directory(directory_path)

Processing songs in: songs_mp3


In [8]:
import pandas as pd

df = pd.DataFrame(
    all_fingerprints,
    columns=["filename", "freq1", "freq2", "delta_time", "anchor_time"]
)

# Preview results
df

,filename,freq1,freq2,delta_time,anchor_time
0,Billie Eilish - BIRDS OF A FEATHER (Official M...,3,3,21,17784
1,Billie Eilish - BIRDS OF A FEATHER (Official M...,3,3,20,18210
2,Billie Eilish - BIRDS OF A FEATHER (Official M...,4,4,20,15066
3,Billie Eilish - BIRDS OF A FEATHER (Official M...,4,4,37,15903
4,Billie Eilish - BIRDS OF A FEATHER (Official M...,4,4,39,17654
...,...,...,...,...,...
14417,Teddy Swims - Lose Control (The Village Sessio...,481,483,15,18785
14418,Teddy Swims - Lose Control (The Village Sessio...,507,508,13,7076
14419,Teddy Swims - Lose Control (The Village Sessio...,508,509,13,18889
14420,Teddy Swims - Lose Control (The Village Sessio...,523,530,24,18795


In [ ]:
df.to_csv("flattened_fingerprints/Flattened_Fingerprints.csv", index=False)

## Get Random Clip from Songs & Process

In [ ]:
import os
import subprocess
import random
import librosa

# Folder setup
input_folder = "songs_mp3"
clip_folder = "clip_mp3"

os.makedirs(clip_folder, exist_ok=True)

def extract_clip(mp3_path, duration=10):
    """Extract a random clip from a full MP3 file."""
    try:
        # Load audio to get total duration
        y, sr = librosa.load(mp3_path, sr=22050)
        total_duration = librosa.get_duration(y=y, sr=sr)

        # Pick random start time
        start_time = 0
        if total_duration > duration:
            start_time = random.uniform(0, total_duration - duration)

        # Output path
        base_name = os.path.splitext(os.path.basename(mp3_path))[0]
        clip_name = f"clip_{base_name}.mp3"
        clip_path = os.path.join(clip_folder, clip_name)

        # FFmpeg command
        cmd = [
            "ffmpeg",
            "-y",                # Overwrite if exists
            "-ss", str(start_time),
            "-t", str(duration),
            "-i", mp3_path,
            "-acodec", "copy",
            clip_path
        ]
        subprocess.run(cmd, check=True)
        print(f"Clip saved: {clip_path}")

    except Exception as e:
        print(f"Error processing {mp3_path}: {e}")

for file in os.listdir(input_folder):
    if file.endswith(".mp3"):
        full_path = os.path.join(input_folder, file)
        extract_clip(full_path, duration=10)

In [ ]:
directory_path = "clip_mp3" 

print(f"Processing songs in: {directory_path}")
all_fingerprints = process_audio_directory(directory_path)

In [ ]:
import pandas as pd

df = pd.DataFrame(
    all_fingerprints,
    columns=["filename", "freq1", "freq2", "delta_time", "anchor_time"]
)

# Preview results
df

In [ ]:
df.to_csv("flattened_fingerprints/Clip_Flattened_Fingerprints.csv", index=False)